In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! pip install SimpleITK

In [ ]:
import SimpleITK as sitk
import matplotlib.pyplot as plt
import os
import numpy as np
import glob

In [ ]:
!pip install tensorflow==2.2.0
!pip install keras==2.4.2
!pip install segmentation_models

In [ ]:
! pip install tensorflow

In [ ]:
import SimpleITK as sitk
import numpy as np
import tensorflow as tf
from keras import layers, models
from sklearn.model_selection import train_test_split
import os
import glob
import cv2

In [ ]:
#Enter modules
import zipfile  # For faster extraction
import keras
import tensorflow as tf
import os
from numpy import expand_dims
import numpy as np # For data manipulation
import matplotlib.pyplot as plt # for draw
from google.colab.patches import cv2_imshow #coordinate cv2 with colab
import nibabel as nib  #To read nii format
import re  # For parsing the filenames (to know their modality)
import glob # For populating the list of files
import imgaug.augmenters as iaa #for data agumention
from sklearn.model_selection import KFold, StratifiedKFold #Data setup and data segmentation
import imgaug as ia
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import CSVLogger
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import time
import datetime
import os
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout
from keras.optimizers import Adam
from keras import backend as K
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.models import load_model
import keras.backend as K
import numpy as np
import tensorflow as tf


In [ ]:
import os
os.environ['SM_FRAMEWORK'] = 'tf.keras'

In [ ]:
from tensorflow import keras
import segmentation_models as sm

In [ ]:
#train data
classified_data = '/content/drive/MyDrive/Classified data'

Z_ADC_paths = []
LABEL_paths = []
class_labels = []

X = []
Y = []
Z = []


def extract_first_three_digits(filepath):
    filename = os.path.basename(filepath)
    match = re.search(r'\d{3}', filename)
    return int(match.group()) if match else float('inf')

for class_index in range(3):
    class_folder = os.path.join(classified_data, f'class_{class_index}')

    Z_ADC_folder = os.path.join(class_folder, 'Z_ADC')
    LABEL_folder = os.path.join(class_folder, 'LABEL')

    Z_ADC_images = os.listdir(Z_ADC_folder)
    LABEL_images = os.listdir(LABEL_folder)

    LABEL_dict = {}

    for LABEL_image in LABEL_images:
        match = re.search(r'(\d+)', LABEL_image)
        if match:
            identifier = match.group(1)
            LABEL_dict[identifier] = os.path.join(LABEL_folder, LABEL_image)

    for Z_ADC_image in Z_ADC_images:
        match = re.search(r'(\d+)', Z_ADC_image)
        if match:
            identifier = match.group(1)


            if identifier in LABEL_dict:
                Z_ADC_paths.append(os.path.join(Z_ADC_folder, Z_ADC_image))
                LABEL_paths.append(LABEL_dict[identifier])
                class_labels.append(class_index)

sorted_indices = np.argsort([extract_first_three_digits(path) for path in Z_ADC_paths])

Z_ADC_paths = np.array(Z_ADC_paths)[sorted_indices]
LABEL_paths = np.array(LABEL_paths)[sorted_indices]
class_labels = np.array(class_labels)[sorted_indices]

print(f"Total matched ADC: {len(Z_ADC_paths)}")
print(f"Total matched LABEL: {len(LABEL_paths)}")
print(f"Class labels: {len(class_labels)}")

X = sitk.ReadImage(Z_ADC_paths.tolist())
Y = sitk.ReadImage(LABEL_paths.tolist())

X = sitk.GetArrayFromImage(X)
Y = sitk.GetArrayFromImage(Y)

X = np.moveaxis(X, 1, -1)
Y = np.moveaxis(Y, 1, -1)

assert len(X) == len(Y)

X = np.array(X)
Y = np.array(Y)
Z = np.array(class_labels)

print(f"X shape (Z_ADC images): {X.shape}")
print(f"Y shape (LABEL images): {Y.shape}")
print(f'Z shape (class_labels): {Z.shape}')


In [ ]:
print(class_labels)

In [ ]:
print(np.unique(class_labels))

In [ ]:
i=0
n_split=5

for train_index, val_index in KFold(n_split).split(X):
  if i==3:
    Z_ADC_train, Z_ADC_val = X[train_index], X[val_index]
    LABEL_train, LABEL_val = Y[train_index], Y[val_index]
    class_labels_train, class_labels_val = Z[train_index], Z[val_index]
  i+=1

print(len(Z_ADC_train), len(Z_ADC_val))
print(len(LABEL_train), len(LABEL_val))
print(len(class_labels_train), len(class_labels_val))

from collections import Counter
print(f'Training class distribution: {Counter(class_labels_train)}')
print(f'Validation class_distribution: {Counter(class_labels_val)}')

In [ ]:
print(class_labels_val)

In [ ]:
print(Z_ADC_train.shape)
print(LABEL_train.shape)
print(Z_ADC_val.shape)
print(LABEL_val.shape)
print(class_labels_train.shape)
print(class_labels_val.shape)

In [ ]:
#test data
classified_data_test = '/content/drive/MyDrive/Classified data_valid'

Z_ADC_test_paths = []
LABEL_test_paths = []
class_labels_test = []

X = []
Y = []
Z = []


def extract_first_three_digits(filepath):
    filename = os.path.basename(filepath)
    match = re.search(r'\d{3}', filename)
    return int(match.group()) if match else float('inf')

for class_test_index in range(3):
    class_test_folder = os.path.join(classified_data_test, f'class_{class_test_index}')

    Z_ADC_test_folder = os.path.join(class_test_folder, 'Z_ADC')
    LABEL_test_folder = os.path.join(class_test_folder, 'LABEL')

    Z_ADC_test_images = os.listdir(Z_ADC_test_folder)
    LABEL_test_images = os.listdir(LABEL_test_folder)

    LABEL_test_dict = {}

    for LABEL_test_image in LABEL_test_images:
        match = re.search(r'(\d+)', LABEL_test_image)
        if match:
            identifier = match.group(1)
            LABEL_test_dict[identifier] = os.path.join(LABEL_test_folder, LABEL_test_image)


    for Z_ADC_test_image in Z_ADC_test_images:
        match = re.search(r'(\d+)', Z_ADC_test_image)
        if match:
            identifier = match.group(1)

            if identifier in LABEL_test_dict:
                Z_ADC_test_paths.append(os.path.join(Z_ADC_test_folder, Z_ADC_test_image))
                LABEL_test_paths.append(LABEL_test_dict[identifier])
                class_labels_test.append(class_test_index)

sorted_indices = np.argsort([extract_first_three_digits(path) for path in Z_ADC_test_paths])


Z_ADC_test_paths = np.array(Z_ADC_test_paths)[sorted_indices]
LABEL_test_paths = np.array(LABEL_test_paths)[sorted_indices]
class_labels_test = np.array(class_labels_test)[sorted_indices]

print(f"Total matched Z_ADC: {len(Z_ADC_test_paths)}")
print(f"Total matched LABEL: {len(LABEL_test_paths)}")
print(f"Class labels: {len(class_labels_test)}")

Z_ADC_test = sitk.ReadImage(Z_ADC_test_paths.tolist())
LABEL_test = sitk.ReadImage(LABEL_test_paths.tolist())

Z_ADC_test = sitk.GetArrayFromImage(Z_ADC_test)
LABEL_test = sitk.GetArrayFromImage(LABEL_test)

Z_ADC_test = np.moveaxis(Z_ADC_test, 1, -1)
LABEL_test = np.moveaxis(LABEL_test, 1, -1)

assert len(Z_ADC_test) == len(LABEL_test)


In [ ]:
print(class_labels_test)

In [ ]:
print(Z_ADC_train.shape)
print(LABEL_train.shape)
print(Z_ADC_val.shape)
print(LABEL_val.shape)
print(Z_ADC_test.shape)
print(LABEL_test.shape)

In [ ]:
# separate slices
Z_ADC_train_slices = []
for i in range(len(Z_ADC_train)):
    for j in range(64):
        Z_ADC_train_img = Z_ADC_train[i][:, :, j]
        Z_ADC_train_img = cv2.resize(Z_ADC_train_img, (128, 128), interpolation=cv2.INTER_NEAREST)
        Z_ADC_train_slices.append(Z_ADC_train_img)


LABEL_train_slices = []
for i in range(len(LABEL_train)):
    for j in range(64):
        LABEL_train_img = LABEL_train[i][:, :, j]
        LABEL_train_img = cv2.resize(LABEL_train_img, (128, 128), interpolation=cv2.INTER_NEAREST)
        LABEL_train_slices.append(LABEL_train_img)

Z_ADC_val_slices = []
for i in range(len(Z_ADC_val)):
    for j in range(64):
        Z_ADC_val_img = Z_ADC_val[i][:, :, j]
        Z_ADC_val_img = cv2.resize(Z_ADC_val_img, (128, 128), interpolation=cv2.INTER_NEAREST)
        Z_ADC_val_slices.append(Z_ADC_val_img)


LABEL_val_slices = []
for i in range(len(LABEL_val)):
    for j in range(64):
        LABEL_val_img = LABEL_val[i][:, :, j]
        LABEL_val_img = cv2.resize(LABEL_val_img, (128, 128), interpolation=cv2.INTER_NEAREST)
        LABEL_val_slices.append(LABEL_val_img)

Z_ADC_test_slices = []
for i in range(len(Z_ADC_test)):
    for j in range(64):
        Z_ADC_test_img = Z_ADC_test[i][:, :, j]
        Z_ADC_test_img = cv2.resize(Z_ADC_test_img, (128, 128), interpolation=cv2.INTER_NEAREST)
        Z_ADC_test_slices.append(Z_ADC_test_img)


LABEL_test_slices = []
for i in range(len(LABEL_test)):
    for j in range(64):
        LABEL_test_img = LABEL_test[i][:, :, j]
        LABEL_test_img = cv2.resize(LABEL_test_img, (128, 128), interpolation=cv2.INTER_NEAREST)
        LABEL_test_slices.append(LABEL_test_img)


In [ ]:
Z_T = []
for i in range(len(class_labels_train)):
  label = class_labels_train[i]
  for j in range(64):  # For each slice in the image
    Z_T.append(label)  # Append the same label for each slice

Z_V = []
for i in range(len(class_labels_val)):
  label = class_labels_val[i]
  for j in range(64):
    Z_V.append(label)

Z_test = []
for i in range(len(class_labels_test)):
  label = class_labels_test[i]
  for j in range(64):
    Z_test.append(label)

In [ ]:
print(len(Z_ADC_train_slices)), print(len(LABEL_train_slices)),print(len(Z_T)), print(len(Z_ADC_val_slices)), print(len(LABEL_val_slices)),  print(len(Z_V)), print(len(Z_ADC_test_slices)), print(len(LABEL_test_slices)), print(len(Z_test))

In [ ]:
X_T = np.array(Z_ADC_train_slices)
Y_T = np.array(LABEL_train_slices)
Z_T = np.array(Z_T)
X_V = np.array(Z_ADC_val_slices)
Y_V = np.array(LABEL_val_slices)
Z_V = np.array(Z_V)
X_test = np.array(Z_ADC_test_slices)
Y_test = np.array(LABEL_test_slices)
Z_test = np.array(Z_test)

In [ ]:
print(X_T.shape , Y_T.shape ,Z_T.shape, X_V.shape, Y_V.shape, Z_V.shape, X_test.shape, Y_test.shape, Z_test.shape)

In [ ]:
np.unique(Y_T[4000])

In [ ]:
np.unique(Y_V[1000])

In [ ]:

np.unique(Y_test[200])

In [ ]:
X_T = X_T.astype(np.float32)
Y_T = Y_T.astype(np.float32)
X_V = X_V.astype(np.float32)
Y_V = Y_V.astype(np.float32)
X_test = X_test.astype(np.float32)
Y_test = Y_test.astype(np.float32)

In [ ]:
Y_T_c = to_categorical(Y_T, 2)

In [ ]:
Y_V_c = to_categorical(Y_V, 2)

In [ ]:
Y_test_c = to_categorical(Y_test, 2)

In [ ]:
X_T.shape, X_V.shape,  X_test.shape, Y_T_c.shape, Y_V_c.shape, Y_test_c.shape

In [ ]:
X_T = np.expand_dims(X_T, axis=-1)
X_V = np.expand_dims(X_V, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)

In [ ]:
X_T.shape, X_V.shape,  X_test.shape, Y_T_c.shape, Y_V_c.shape, Y_test_c.shape

In [ ]:
X_T = X_T.astype(np.float32)
Y_T = Y_T.astype(np.float32)
X_V = X_V.astype(np.float32)
Y_V = Y_V.astype(np.float32)
X_test = X_test.astype(np.float32)
Y_test = Y_test.astype(np.float32)

In [ ]:
# Data_generator_train(using the RAM in the correct way)
import random
def Data_generator_train(X,y, batch_size=8):
    while True:
          for start in range(0, X.shape[0], batch_size):
              x_batch = []
              y_batch = []
              end = min(start + batch_size, X.shape[0])
              ids_batch = X[start:end]
              for idd in range(start,end):
                   a=random.randint(0,2)
                   if a==0:
                      image =np.fliplr(X[idd])
                      label =np.fliplr(y[idd])
                   if a==1:
                     image =np.flipud(X[idd])
                     label =np.flipud(y[idd])
                   elif a==2:
                     image =(X[idd])
                     label =(y[idd])



                   x_batch.append(image)
                   y_batch.append(label)
              x_batch= np.array(x_batch)
              y_batch= np.array(y_batch)
              yield x_batch, y_batch


In [ ]:
m,b=next(iter(Data_generator_train(X_T,Y_T_c)))

print(m.shape)
print(b.shape)

In [ ]:
import numpy as np

def Data_generator_test(X, y, batch_size=1):
    while True:
        for start in range(0, X.shape[0], batch_size):
            x_batch = []
            y_batch = []
            end = min(start + batch_size, X.shape[0])
            ids_batch = X[start:end]


            for idd in range(start, end):
                image = X[idd]
                label = y[idd]

                x_batch.append(image)
                y_batch.append(label)


            x_batch = np.array(x_batch)
            y_batch = np.array(y_batch)


            yield x_batch, y_batch


In [ ]:
m,b=next(iter(Data_generator_test(X_V,Y_V_c)))

print(m.shape)
print(b.shape)

In [ ]:
m,b=next(iter(Data_generator_test(X_test,Y_test_c)))

print(m.shape)
print(b.shape)

In [ ]:
X_V_class_0 = []
X_V_class_1 = []
X_V_class_2 = []

Y_V_c_class_0 = []
Y_V_c_class_1 = []
Y_V_c_class_2 = []

for i in range(len(Z_V)):
    if Z_V[i] == 0:
        X_V_class_0.append(X_V[i])
        Y_V_c_class_0.append(Y_V_c[i])
    elif Z_V[i] == 1:
        X_V_class_1.append(X_V[i])
        Y_V_c_class_1.append(Y_V_c[i])
    elif Z_V[i] == 2:
        X_V_class_2.append(X_V[i])
        Y_V_c_class_2.append(Y_V_c[i])

In [ ]:
X_V_class_0 = np.array(X_V_class_0)
Y_V_c_class_0 = np.array(Y_V_c_class_0)
X_V_class_1 = np.array(X_V_class_1)
Y_V_c_class_1 = np.array(Y_V_c_class_1)
X_V_class_2 = np.array(X_V_class_2)
Y_V_c_class_2 = np.array(Y_V_c_class_2)

print("Shape of X_V_class_0:", X_V_class_0.shape)
print("Shape of Y_V_c_class_0:", Y_V_c_class_0.shape)
print("Shape of X_V_class_1:", X_V_class_1.shape)
print("Shape of Y_V_c_class_1:", Y_V_c_class_1.shape)
print("Shape of X_V_class_2:", X_V_class_2.shape)
print("Shape of Y_V_c_class_2:", Y_V_c_class_2.shape)

In [ ]:
X_test_class_0 = []
X_test_class_1 = []
X_test_class_2 = []

Y_test_c_class_0 = []
Y_test_c_class_1 = []
Y_test_c_class_2 = []

for i in range(len(Z_test)):
  if Z_test[i] == 0:
    X_test_class_0.append(X_test[i])
    Y_test_c_class_0.append(Y_test_c[i])
  elif Z_test[i] == 1:
    X_test_class_1.append(X_test[i])
    Y_test_c_class_1.append(Y_test_c[i])
  elif Z_test[i] == 2:
    X_test_class_2.append(X_test[i])
    Y_test_c_class_2.append(Y_test_c[i])

In [ ]:
X_test_class_0 = np.array(X_test_class_0)
Y_test_c_class_0 = np.array(Y_test_c_class_0)
X_test_class_1 = np.array(X_test_class_1)
Y_test_c_class_1 = np.array(Y_test_c_class_1)
X_test_class_2 = np.array(X_test_class_2)
Y_test_c_class_2 = np.array(Y_test_c_class_2)

print("Shape of X_test_class_0:", X_test_class_0.shape)
print("Shape of Y_test_c_class_0:", Y_test_c_class_0.shape)
print("Shape of X_test_class_1:", X_test_class_1.shape)
print("Shape of Y_test_c_class_1:", Y_test_c_class_1.shape)
print("Shape of X_test_class_2:", X_test_class_2.shape)
print("Shape of Y_test_c_class_2:", Y_test_c_class_2.shape)

In [ ]:
model= sm.Unet('vgg16', classes=2, activation='softmax', encoder_weights=None, input_shape=(128, 128, 1))
model.summary()

In [ ]:
dice_loss = sm.losses.DiceLoss()
focal_loss = sm.losses.CategoricalFocalLoss()
total_loss = dice_loss + (1 * focal_loss)

metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5), 'accuracy']

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss=dice_loss + (1 * focal_loss),
    metrics=[sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5), 'accuracy'],
    # No need for loss_weights in a single-output model
)


In [ ]:
from keras.callbacks import EarlyStopping,ModelCheckpoint,CSVLogger

model_checkpoint = ModelCheckpoint('/content/drive/MyDrive/ZADC_models/U_net_resnet34/fold4.weights.h5', save_best_only=True,save_weights_only=True, verbose=2, monitor='val_f1-score',mode='max')
csv_logger = CSVLogger('/content/drive/MyDrive/ZADC_models/U_net_resnet34/fold4.weights.csv', append=True)

In [ ]:
def step_decay(epoch):

     initAlpha = 0.0001   #equals lr
     factor = .9    #based on testing
     dropEvery = 10    #based on testing
     alpha = initAlpha * (factor ** np.floor((1 + epoch) / dropEvery))
     return float(alpha)

In [ ]:
batch_size =8
TrainSteps = int(len(X_T)  / batch_size)
ValSteps   = int(len(X_V))

In [ ]:
Y_T_c = Y_T_c.astype('float32')
Y_V_c = Y_V_c.astype('float32')

In [ ]:
# LearningRateScheduler adjusts the learning rate
%%time
from keras.callbacks import LearningRateScheduler

history = model.fit(Data_generator_train(X_T, Y_T_c),
                    epochs = 70,
                    steps_per_epoch=TrainSteps,
                    validation_data = Data_generator_test(X_V, Y_V_c),
                    validation_steps = ValSteps,
                    callbacks=[model_checkpoint,csv_logger, LearningRateScheduler(step_decay)],
                    shuffle=False)

In [ ]:
model.load_weights('/content/drive/MyDrive/ZADC_models/U_net_vgg16/fold4.weights.h5')

In [ ]:
#overall score
scores=model.evaluate(X_V,Y_V_c , batch_size=1)
print("Loss: {:.5}".format(scores[0]))
for metric, value in zip(metrics, scores[1:]):
    print("mean {}: {:.5}".format(metric, value))

In [ ]:
#class_0 score
scores=model.evaluate(X_V_class_0, Y_V_c_class_0 , batch_size=1)
print("Loss: {:.5}".format(scores[0]))
for metric, value in zip(metrics, scores[1:]):
    print("mean {}: {:.5}".format(metric, value))

In [ ]:
#class_1 score
scores=model.evaluate(X_V_class_1, Y_V_c_class_1 , batch_size=1)
print("Loss: {:.5}".format(scores[0]))
for metric, value in zip(metrics, scores[1:]):
    print("mean {}: {:.5}".format(metric, value))

In [ ]:
#class_2 score
scores=model.evaluate(X_V_class_2, Y_V_c_class_2 , batch_size=1)
print("Loss: {:.5}".format(scores[0]))
for metric, value in zip(metrics, scores[1:]):
    print("mean {}: {:.5}".format(metric, value))

In [ ]:
# validation prediction:
p=model.predict(X_V)
y=Y_V_c

p_class_0 = model.predict(X_V_class_0)
y_class_0 = Y_V_c_class_0

p_class_1 = model.predict(X_V_class_1)
y_class_1 = Y_V_c_class_1

p_class_2 = model.predict(X_V_class_2)
y_class_2 = Y_V_c_class_2

In [ ]:
pred=np.argmax(p, axis=-1)
ytrue=np.argmax(y, axis=-1)

pred_class_0=np.argmax(p_class_0, axis=-1)
ytrue_class_0=np.argmax(y_class_0, axis=-1)

pred_class_1=np.argmax(p_class_1, axis=-1)
ytrue_class_1=np.argmax(y_class_1, axis=-1)

pred_class_2=np.argmax(p_class_2, axis=-1)
ytrue_class_2=np.argmax(y_class_2, axis=-1)

In [ ]:
Pred=pred.flatten()
Ytrue=ytrue.flatten()

Pred_class_0=pred_class_0.flatten()
Ytrue_class_0=ytrue_class_0.flatten()

Pred_class_1=pred_class_1.flatten()
Ytrue_class_1=ytrue_class_1.flatten()

Pred_class_2=pred_class_2.flatten()
Ytrue_class_2=ytrue_class_2.flatten()

In [ ]:
print(Ytrue.shape)
print(Ytrue_class_0.shape)
print(Pred_class_1.shape)
print(Ytrue_class_2.shape)

In [ ]:
from sklearn.metrics import classification_report
print(f'classification_report:')
print(f'overall:')
print(classification_report(Pred, Ytrue, digits=5))

print(f'class_0:')
print(classification_report(Pred_class_0, Ytrue_class_0, digits=5))

print(f'class_1:')
print(classification_report(Pred_class_1, Ytrue_class_1, digits=5))

print(f'class_2:')
print(classification_report(Pred_class_2, Ytrue_class_2, digits=5))

In [ ]:
#overall score
scores=model.evaluate(X_test,Y_test_c , batch_size=1)
print("Loss: {:.5}".format(scores[0]))
for metric, value in zip(metrics, scores[1:]):
    print("mean {}: {:.5}".format(metric, value))

In [ ]:
#class_0 score
scores=model.evaluate(X_test_class_0, Y_test_c_class_0 , batch_size=1)
print("Loss: {:.5}".format(scores[0]))
for metric, value in zip(metrics, scores[1:]):
    print("mean {}: {:.5}".format(metric, value))

In [ ]:
#class_1 score
scores=model.evaluate(X_test_class_1, Y_test_c_class_1 , batch_size=1)
print("Loss: {:.5}".format(scores[0]))
for metric, value in zip(metrics, scores[1:]):
    print("mean {}: {:.5}".format(metric, value))

In [ ]:
#class_2 score
scores=model.evaluate(X_test_class_2, Y_test_c_class_2 , batch_size=1)
print("Loss: {:.5}".format(scores[0]))
for metric, value in zip(metrics, scores[1:]):
    print("mean {}: {:.5}".format(metric, value))

In [ ]:
# test prediction:
p=model.predict(X_test)
y=Y_test_c

p_class_0 = model.predict(X_test_class_0)
y_class_0 = Y_test_c_class_0

p_class_1 = model.predict(X_test_class_1)
y_class_1 = Y_test_c_class_1

p_class_2 = model.predict(X_test_class_2)
y_class_2 = Y_test_c_class_2

In [ ]:
pred=np.argmax(p, axis=-1)
ytrue=np.argmax(y, axis=-1)

pred_class_0=np.argmax(p_class_0, axis=-1)
ytrue_class_0=np.argmax(y_class_0, axis=-1)

pred_class_1=np.argmax(p_class_1, axis=-1)
ytrue_class_1=np.argmax(y_class_1, axis=-1)

pred_class_2=np.argmax(p_class_2, axis=-1)
ytrue_class_2=np.argmax(y_class_2, axis=-1)

In [ ]:
Pred=pred.flatten()
Ytrue=ytrue.flatten()

Pred_class_0=pred_class_0.flatten()
Ytrue_class_0=ytrue_class_0.flatten()

Pred_class_1=pred_class_1.flatten()
Ytrue_class_1=ytrue_class_1.flatten()

Pred_class_2=pred_class_2.flatten()
Ytrue_class_2=ytrue_class_2.flatten()

In [ ]:
print(Ytrue.shape)
print(Ytrue_class_0.shape)
print(Pred_class_1.shape)
print(Ytrue_class_2.shape)

In [ ]:
from sklearn.metrics import classification_report
print(f'classification_report:')
print(f'overall:')
print(classification_report(Pred, Ytrue, digits=5))

print(f'class_0:')
print(classification_report(Pred_class_0, Ytrue_class_0, digits=5))

print(f'class_1:')
print(classification_report(Pred_class_1, Ytrue_class_1, digits=5))

print(f'class_2:')
print(classification_report(Pred_class_2, Ytrue_class_2, digits=5))